In [ ]:
%run /home/bb/src/python/valkyrie/lib/valkyrie/apps/common/run_notebook_import.py

In [ ]:
sdate, edate = '2023-01-01', '2023-03-31'
instrs = ['BDM.BTC.USDT.FP']
freq = '1s'
ret_n_s = [60, 120, 300, 600]
root_dir = '/home/bb/data/BDM'
#########################################################
#DataMgr
#########################################################
channels = ['bpx_last','apx_last'] + ['bq_last','aq_last','buy_qty_sum', 'sell_qty_sum'] 
n_channels = len(channels)
H, W = 16, 64


data_mgr = DataMgr(sdate, edate, freq, root_dir, instrs = instrs, ret_n_s = ret_n_s)
df_res = data_mgr.get(instrs[0]) 
df2t2 = Df2T2(df_res, H, W, xcols = channels, ycol = 'mid_last_ret_300_n', yscaler = 1)

train_set, val_set = SubDataSet.create_train_valiation_pair(df2t2, 0.8)
dm = DataModule.from_dataset(train_set, val_set, 2048)

# Mul = 1

In [ ]:
alex_reg_net1 = AlexRegNet(lr = 1e-4, num_channels = n_channels, height = H, down_mul = 1)
X = df2t2[0][0]
ml_utils.layer_summary(alex_reg_net1, X.shape)
#ml_utils.parameter_summary(testNet)

In [ ]:
trainer = Trainer(max_epochs=100, log_fn = 'alex_300s')
with Timer('training'):
    trainer.fit(alex_reg_net1, dm)

In [ ]:
ne = NetEvaluator(alex_reg_net1, dm)
ne.eval_performance()